# Moscow housing prediction [Short]

Written by<br> 

Hauk Aleksander Olaussen - **Student ID: *504903*** <br>

Charbel Badr - **Student ID: *557389***<br>

Noran Baskaran - **Student ID: *504877*** <br>

Kaggle competition name: **Moscow housing**

Kaggle team name: **Team 69**

## Introduction

This short notebook will contain the code and comments for you to reproduce our best performing model on the public testset. It will make use of the `Preprocessor.py` and `TestModel.py` classes, just as in **long.ipynb**. We will not do any exploration or engineering in this notebook other than run the functions actually doing this job in `Preprocessor.py`. We hope that this will contribute to a readable model with as little "code gore" as possible. 

In [35]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
%autoreload
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from Preprocessor import Preprocessor
from TestModel import TestModel
import lightgbm as lgbm
import pandas as pd
import numpy as np
import xgboost

preprocessor = Preprocessor()

## Extracting the data

As the data is contained within *.csv* files, we need to read this and put it into a `pandas` DataFrame. This will make it easier to perform the preprocessing later. This is done in the same way as you saw in the long notebook.

In [37]:
merged_final = pd.concat([preprocessor.merged.copy(), preprocessor.merged_test.copy()], ignore_index=True)
print(f"The dataframe now has {len(merged_final)} entries")

The dataframe now has 33222 entries


Lets run the preprocessing functions

In [38]:
general_removed = preprocessor.general_removal(merged_final.copy()) # Removes "layout", "ceiling", "balconies", "loggias", "elevator_without", "street", "address"
data = preprocessor.remove_NaNs(general_removed.copy())             # Fills in NaN valiues in different ways depending on the entry
data = preprocessor.impute_nans_with_building_id(data.copy(), ["condition"]) # Fills in NaNs for condition
# Removing outliers in lat-lon, as some of them are very far away
is_outlier = (data["longitude"] > 55) | (data["latitude"] < 54)
outliers = data.copy()[is_outlier]
data = preprocessor.fix_latlon_outliers(data.copy(), outliers)
data.head()

,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,bathrooms_shared,bathrooms_private,...,longitude,district,constructed,material,stories,elevator_passenger,elevator_service,parking,garbage_chute,heating
0,0,3.0,7139520.0,59.2,12.5,31.0,2.0,2.0,0.0,2.0,...,37.478055,11.0,2021.0,3.0,9.0,1.0,1.0,1.0,-1.0,-1.0
1,1,-1.0,10500000.0,88.0,14.2,48.0,18.0,3.0,2.0,0.0,...,37.666647,2.0,2010.0,3.0,25.0,1.0,1.0,1.0,-1.0,0.0
2,2,3.0,9019650.0,78.5,22.5,40.8,12.0,3.0,0.0,2.0,...,37.515335,6.0,2021.0,3.0,15.0,1.0,1.0,1.0,-1.0,-1.0
3,3,-1.0,10500000.0,88.0,14.0,48.0,18.0,3.0,0.0,2.0,...,37.666647,2.0,2010.0,3.0,25.0,1.0,1.0,1.0,-1.0,0.0
4,4,-1.0,13900000.0,78.0,17.0,35.0,7.0,2.0,1.0,0.0,...,37.451438,11.0,2017.0,2.0,15.0,1.0,1.0,1.0,0.0,0.0


## Adding features

Now lets start adding the features we want for the model. We start with the distance features

In [39]:
# Adding distance features
distance_df = preprocessor.combine_latlon_subway(data.copy(), read_from_file=True)  # Subway
distance_df = preprocessor.closest_hospital(distance_df.copy())                     # Hospital
distance_df = preprocessor.closest_park(distance_df.copy())                         # Parks 
distance_df = preprocessor.distance_luxury_village(distance_df.copy())              # Luxury village
distance_df = preprocessor.closest_uni(distance_df.copy())                          # University
distance_df = preprocessor.inside_golden_mile(distance_df.copy())                   # Golden mile, boolean
distance_df = preprocessor.distance_from_golden_mile(distance_df.copy())            # Golden mile, distance
distance_df = preprocessor.inside_khamovniki(distance_df.copy())                    # Khamovniki, boolean
distance_df = preprocessor.distance_from_khamovniki_center(distance_df.copy())      # Khamovniki, distance
distance_df = preprocessor.combine_latlon(distance_df.copy())                       # Red square
distance_df = preprocessor.closest_powerplant(distance_df.copy())                   # Power plant
print("Distance features added")

Distance features added


We continue by adding combined features.

In [40]:
# Creating the new features
featured = preprocessor.combine_windows(distance_df.copy(), boolean=True) # has_windows feature
featured = preprocessor.redo_new(featured.copy())                         # Fixing new feature before combining
featured = preprocessor.combine_new_constructed_distance(featured.copy()) # scaled_constructed feature
featured = preprocessor.floor_fraction(featured.copy())                   # floor_fraction feature
featured = preprocessor.combine_floor_stories(featured.copy())            # floor_stories feature
featured = preprocessor.area_distance(featured.copy())                    # area_distance feature
featured = preprocessor.combine_baths(featured.copy())                    # bathroom_amount feature
featured.head()

,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,bathrooms_shared,bathrooms_private,...,distance_golden_mile,inside_khamovniki,distance_khamovniki_center,distance_center,closest_powerplant,has_windows,scaled_constructed,floor_fraction,floor_stories,area_distance
0,0,3.0,7139520.0,59.2,12.5,31.0,2.0,2.0,0.0,2.0,...,22.936124,0,21.990803,25.022080,10.364623,True,2021.000000,0.222222,18.0,0.171074
1,1,-1.0,10500000.0,88.0,14.2,48.0,18.0,3.0,2.0,0.0,...,14.282081,0,15.264384,12.267029,7.957621,True,1999.059871,0.720000,450.0,0.106986
2,2,3.0,9019650.0,78.5,22.5,40.8,12.0,3.0,0.0,2.0,...,9.839135,0,8.719216,12.060134,6.032235,True,2021.000000,0.800000,180.0,0.119716
3,3,-1.0,10500000.0,88.0,14.0,48.0,18.0,3.0,0.0,2.0,...,14.282081,0,15.264384,12.267029,7.957621,True,1999.059871,0.720000,450.0,0.106986
4,4,-1.0,13900000.0,78.0,17.0,35.0,7.0,2.0,1.0,0.0,...,18.839560,0,17.732445,21.041782,11.287607,True,2013.007917,0.466667,105.0,0.127619


## Removing features

Lets now remove features we do not indend to keep.

In [41]:
to_be_removed = ["rooms", "floor", "garbage_chute", "seller", "phones", "area_kitchen", "area_living", "id", "constructed", "bathrooms_private", "bathrooms_shared", "new", "windows_street", "stories", "windows_court", "elevator_passenger", "elevator_service"]
removed = preprocessor.remove_labels(featured.copy(), to_be_removed)
removed.head()

,price,area_total,condition,building_id,latitude,longitude,district,material,parking,heating,...,distance_golden_mile,inside_khamovniki,distance_khamovniki_center,distance_center,closest_powerplant,has_windows,scaled_constructed,floor_fraction,floor_stories,area_distance
0,7139520.0,59.2,-1.0,4076,55.544046,37.478055,11.0,3.0,1.0,-1.0,...,22.936124,0,21.990803,25.022080,10.364623,True,2021.000000,0.222222,18.0,0.171074
1,10500000.0,88.0,3.0,1893,55.861282,37.666647,2.0,3.0,1.0,0.0,...,14.282081,0,15.264384,12.267029,7.957621,True,1999.059871,0.720000,450.0,0.106986
2,9019650.0,78.5,-1.0,5176,55.663299,37.515335,6.0,3.0,1.0,-1.0,...,9.839135,0,8.719216,12.060134,6.032235,True,2021.000000,0.800000,180.0,0.119716
3,10500000.0,88.0,2.0,1893,55.861282,37.666647,2.0,3.0,1.0,0.0,...,14.282081,0,15.264384,12.267029,7.957621,True,1999.059871,0.720000,450.0,0.106986
4,13900000.0,78.0,3.0,6604,55.590785,37.451438,11.0,2.0,1.0,0.0,...,18.839560,0,17.732445,21.041782,11.287607,True,2013.007917,0.466667,105.0,0.127619


## Logifying skewed data

We logify some of the data, as they are quite skewed originally.

In [42]:
finished = preprocessor.log10ify(removed.copy(), "area_total")
finished = preprocessor.log10ify(finished.copy(), "price")
finished.head()

,price,area_total,condition,building_id,latitude,longitude,district,material,parking,heating,...,distance_golden_mile,inside_khamovniki,distance_khamovniki_center,distance_center,closest_powerplant,has_windows,scaled_constructed,floor_fraction,floor_stories,area_distance
0,6.853669,1.772322,-1.0,4076,55.544046,37.478055,11.0,3.0,1.0,-1.0,...,22.936124,0,21.990803,25.022080,10.364623,True,2021.000000,0.222222,18.0,0.171074
1,7.021189,1.944483,3.0,1893,55.861282,37.666647,2.0,3.0,1.0,0.0,...,14.282081,0,15.264384,12.267029,7.957621,True,1999.059871,0.720000,450.0,0.106986
2,6.955190,1.894870,-1.0,5176,55.663299,37.515335,6.0,3.0,1.0,-1.0,...,9.839135,0,8.719216,12.060134,6.032235,True,2021.000000,0.800000,180.0,0.119716
3,7.021189,1.944483,2.0,1893,55.861282,37.666647,2.0,3.0,1.0,0.0,...,14.282081,0,15.264384,12.267029,7.957621,True,1999.059871,0.720000,450.0,0.106986
4,7.143015,1.892095,3.0,6604,55.590785,37.451438,11.0,2.0,1.0,0.0,...,18.839560,0,17.732445,21.041782,11.287607,True,2013.007917,0.466667,105.0,0.127619


## Preparing for the model

We will now split the finished DataFrame back into training and test data. The training data contains prices, which we have put into a variable called *labels* above. The test data does not have prices - as this is the task at hand. We will use a combination of *XGBoost* and *LGBM* for our prediction. The configurations have been found using Microsofts autoML library *flaml*.

#### XGBoost

In [55]:
train_data = finished[:23285].copy()                 # Data used for fitting
test_data = finished[23285:].copy()                  # Actual data we want to predict

train_data = preprocessor.fix_categorical(train_data.copy()) # Changes the type of categorical data to "category" instead of float
test_data = preprocessor.fix_categorical(test_data.copy())   # Changes the type of categorical data to "category" instead of float

train_area = data[:23285]["area_total"]              # Extracting the area_total from data. It will be used to convert back to actual prices later
test_area = data[23285:]["area_total"]               # Same as above, but for test set

labels = data[:23285]["price"] / train_area          # Learning labels

train_data.drop("price", 1, inplace=True)            # Removing labels from training set
test_data.drop("price", 1, inplace=True)             # Removing labels from training set, even though they are all NaN'
print("Number of features:", train_data.shape[1])

Number of features: 25


In [56]:
xgboost_model = TestModel(train_data, labels)
xgb_config = {'n_estimators': 88, 'max_leaves': 162, 'min_child_weight': 5.0049737297246, 'learning_rate': 0.14553401958360282, 'subsample': 0.8552583010104474, 'colsample_bylevel': 0.7585765039287123, 'colsample_bytree': 0.6515117665948414, 'reg_alpha': 0.25929972281749397, 'reg_lambda': 0.0789164449038185}
xgb_model = xgboost_model.xgboost_model(xgb_config)
xgb_pipe = Pipeline([("scaler", StandardScaler()), ("xbg", xgb_model)])

In [57]:
print("Fitting XGBoost model...")
xgb_fitted = xgb_pipe.fit(train_data, labels)
print("XGBoost fitting complete!")

Fitting XGBoost model...
XGBoost fitting complete!


In [58]:
xgb_prediction = xgb_fitted.predict(test_data)
xgb_predicted_prices = xgb_prediction * test_area
print("XGBoost prediction finished!")
xgb_predicted_prices

XGBoost prediction finished!


23285    2.765177e+07
23286    1.025092e+07
23287    6.306911e+06
23288    8.238717e+06
23289    4.827186e+06
             ...     
33217    2.885561e+07
33218    2.016149e+07
33219    9.385647e+06
33220    8.992284e+06
33221    8.509962e+06
Name: area_total, Length: 9937, dtype: float64

#### LGBM

In [59]:
lgbm_model = TestModel(train_data, labels)
lgbm_config = {'n_estimators': 1025, 'num_leaves': 89, 'min_child_samples': 6, 'learning_rate': 0.07207849734387042, 'log_max_bin': 7, 'colsample_bytree': 0.5911916499145919, 'reg_alpha': 0.0009765625, 'reg_lambda': 0.017810670616715204}
lgbm_model = lgbm_model.lgbm_model(lgbm_config)
lgbm_pipe = Pipeline([("scaler", StandardScaler()), ("lgbm", lgbm_model)])

In [61]:
print("Fitting LGBM model...")
lgbm_fitted = lgbm_pipe.fit(train_data, labels)
print("LGBM fitting complete!")

Fitting LGBM model...
LGBM fitting complete!


In [62]:
lgbm_prediction = lgbm_fitted.predict(test_data)
lgbm_predicted_prices = lgbm_prediction * test_area
print("LGBM  prediction finished!")
lgbm_predicted_prices

LGBM  prediction finished!


23285    2.702101e+07
23286    1.008016e+07
23287    6.298896e+06
23288    8.271902e+06
23289    5.124773e+06
             ...     
33217    2.876144e+07
33218    2.115026e+07
33219    9.908472e+06
33220    9.950100e+06
33221    5.818801e+06
Name: area_total, Length: 9937, dtype: float64

#### Combining predictions

We will now combined the results from these two by taking the average value they predicted

In [63]:
combined_prediction = (xgb_predicted_prices + lgbm_predicted_prices) / 2
xgboost_model.save_predictions(list(combined_prediction))

,id,price_prediction
0,23285,2.733639e+07
1,23286,1.016554e+07
2,23287,6.302904e+06
3,23288,8.255309e+06
4,23289,4.975979e+06
...,...,...
9932,33217,2.880853e+07
9933,33218,2.065587e+07
9934,33219,9.647060e+06
9935,33220,9.471192e+06


# Current

best model xgboost configs {'n_estimators': 911, 'max_leaves': 275, 'min_child_weight': 8.554398319481574, 'learning_rate': 0.13797523454204058, 'subsample': 0.7689953576765899, 'colsample_bylevel': 0.3710959011675493, 'colsample_bytree': 0.4323890189027554, 'reg_alpha': 0.016828526535595058, 'reg_lambda': 2.800495230922604} 

best model lgbm configs {'n_estimators': 1025, 'num_leaves': 89, 'min_child_samples': 6, 'learning_rate': 0.07207849734387042, 'log_max_bin': 7, 'colsample_bytree': 0.5911916499145919, 'reg_alpha': 0.0009765625, 'reg_lambda': 0.017810670616715204}

In [51]:
train_data = finished[:23285].copy()
test_data = finished[23285:].copy()
test_data.drop("price", 1, inplace=True)


train_area = data[:23285]["area_total"]
test_area = data[23285:]["area_total"]

labels = data[:23285]["price"] / train_area 

# Splitting training data into training and validation, removed the price for each of them afterwards
x_train, x_test, y_train, y_test = train_test_split(train_data, labels, stratify=train_data.price.round(), test_size=0.0002)
x_train.drop("price", 1, inplace=True)
x_test.drop("price", 1, inplace=True)
x_train = preprocessor.fix_categorical(x_train.copy()) # Changes the type of categorical data to "category" instead of float
x_test = preprocessor.fix_categorical(x_test.copy()) # Changes the type of categorical data to "category" instead of float

print(f"Number of features for LGBM:", x_train.shape[1])

Number of features for LGBM: 25


In [52]:
xgboost_model = TestModel(y_train, x_train)
best_xgboost = xgboost_model.autoMLfit(x_train, y_train, ["xgboost"], time=60, metric="mse")
xgboost_pred = [pred for pred in xgboost_model.autoMLpredict(x_test)] * (10 ** x_test["area_total"])

lgbm_model = TestModel(y_train, x_train)
best_lgbm = lgbm_model.autoMLfit(x_train, y_train, ["lgbm"], time=60, metric="mse")
lgbm_pred = [pred for pred in lgbm_model.autoMLpredict(x_test)] * (10 ** x_test["area_total"])

[flaml.automl: 11-19 04:15:08] {1485} INFO - Data split method: uniform
[flaml.automl: 11-19 04:15:08] {1489} INFO - Evaluation method: holdout
[flaml.automl: 11-19 04:15:08] {1540} INFO - Minimizing error metric: mse
[flaml.automl: 11-19 04:15:08] {1577} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 11-19 04:15:08] {1826} INFO - iteration 0, current learner xgboost
[flaml.automl: 11-19 04:15:08] {1943} INFO - Estimated sufficient time budget=505s. Estimated necessary time budget=1s.
[flaml.automl: 11-19 04:15:08] {2023} INFO -  at 0.2s,	estimator xgboost's best error=45490703755.2261,	best estimator xgboost's best error=45490703755.2261
[flaml.automl: 11-19 04:15:08] {1826} INFO - iteration 1, current learner xgboost
[flaml.automl: 11-19 04:15:08] {2023} INFO -  at 0.3s,	estimator xgboost's best error=45490703755.2261,	best estimator xgboost's best error=45490703755.2261
[flaml.automl: 11-19 04:15:08] {1826} INFO - iteration 2, current learner xgboost
[flaml.aut

In [53]:
xgboost_model.autoML_print_best_model()
lgbm_model.autoML_print_best_model()

best model xgboost
configs {'n_estimators': 88, 'max_leaves': 162, 'min_child_weight': 5.0049737297246, 'learning_rate': 0.14553401958360282, 'subsample': 0.8552583010104474, 'colsample_bylevel': 0.7585765039287123, 'colsample_bytree': 0.6515117665948414, 'reg_alpha': 0.25929972281749397, 'reg_lambda': 0.0789164449038185}
best model lgbm
configs {'n_estimators': 1025, 'num_leaves': 89, 'min_child_samples': 6, 'learning_rate': 0.07207849734387042, 'log_max_bin': 7, 'colsample_bytree': 0.5911916499145919, 'reg_alpha': 0.0009765625, 'reg_lambda': 0.017810670616715204}


In [54]:
combined = (xgboost_pred + lgbm_pred) / 2
test_labels = [lab for lab in y_test] * (10 ** x_test["area_total"])

print("RMSLE for XGBoost model: %s" % xgboost_model.root_mean_squared_log_error(test_labels, xgboost_pred))
print("RMSLE for LGBM model: %s\n" % xgboost_model.root_mean_squared_log_error(test_labels, lgbm_pred))

print("RMSLE for combined models: %s\n" % xgboost_model.root_mean_squared_log_error(test_labels, combined))

RMSLE for XGBoost model: 0.08754455752444734
RMSLE for LGBM model: 0.06752396455762834

RMSLE for combined models: 0.0685114987210302

